Jordan Picone's project

In [1]:
import json
import pandas as pd
import nltk
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
def isRes(ID):
        if ID in res_tag:
                return True
        else:
                return False

def cuisine(cat):
        # finish these up before analysis
        # To find these labels I filtered all businesses tagged as restaurants and made a set of 
        # other tags associated with those restaurants. Out of those 343 I chose the labels manually
        american = ['American (New)', 'American', 'American (Traditional)', 'Southern']
        middle_east = ['Moroccan', 'Egyptian', 'Middle Eastern', 'Turkish', 'Halal', 'Pakistani', 'Persian/Iranian', 'Arabian',]
        latin_america = ['Latin American', 'Peruvian', 'Venezuelan', 'Salvadoran', 'Mexican', 'Brazilian', 'Colombian', 'Puerto Rican']
        east_asia = ['Asian', 'Taiwanese', 'Cambodian', 'Izakaya', 'Singaporean', 'Thai', 'Hong Kong Style Cafe', 'Korean', 'Japanese', 'Chinese', 'Laotian', 'Szechuan', 'Cantonese', 'Vietnamese']
        if cat == None:
            return None
        x = cat.split(', ')
        for i in x:
            if i in middle_east:
                return 'me'
            if i in american:
                return 'am'
            if i in latin_america:
                return 'la'
            if i in east_asia:
                return 'as'
            else:
                return None

Read in businesses and filter out the ones we want

In [3]:
fn = r'yelp_dataset\yelp_academic_dataset_business.json'
res = pd.read_json(fn, lines=True)
res['restaurant'] = res['categories'].map(lambda x: cuisine(x))
res = res.dropna()

In [4]:
res['restaurant'].value_counts()

as    2787
am    2327
la    1400
me     614
Name: restaurant, dtype: int64

In [5]:
res = res.dropna(subset = ['restaurant'])

In [6]:
res.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,restaurant
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,1,"{'Ambience': '{'touristy': False, 'hipster': F...","Persian/Iranian, Turkish, Middle Eastern, Rest...","{'Tuesday': '12:0-21:0', 'Wednesday': '12:0-21...",me
44,Ga2Bt7xfqoggTypWD5VpoQ,Amando's Bros,2602 W Southern Ave,Tempe,AZ,85282,33.393199,-111.976270,4.0,9,0,"{'Caters': 'False', 'RestaurantsGoodForGroups'...","Mexican, Restaurants","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",la
73,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,Toronto,ON,M4M 1Y3,43.666376,-79.348773,4.0,116,1,"{'RestaurantsTakeOut': 'True', 'Alcohol': 'u'b...","Vietnamese, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",as
94,Ir_QIzs-4o9ElOtiGuxJrw,Torteria Lupita,4601 Rue Notre-Dame O,Montréal,QC,H4C 1S5,45.473632,-73.588135,4.0,19,0,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Mexican, Sandwiches, Restaurants, Breakfast & ...","{'Tuesday': '12:0-20:0', 'Wednesday': '12:0-20...",la
137,t749-3jKKDyO8QH3VezUHA,Chao Phraya,50 Avenue Laurier O,Montréal,QC,H2T 2N4,45.522376,-73.593123,3.5,48,0,"{'RestaurantsReservations': 'True', 'Ambience'...","Thai, Restaurants","{'Tuesday': '17:0-22:0', 'Wednesday': '17:0-22...",as


In [7]:
# read in the reviews
fn = r'yelp_dataset\yelp_academic_dataset_review.json'
ChunkSize = 100000
# This is a weird way to do it, but it's necessary with such a huge file.
for chunk in pd.read_json(fn, chunksize=ChunkSize, lines=True):
    df = chunk
    break

In [8]:
res[['business_id', 'restaurant']]

,business_id,restaurant
30,9sRGfSVEfLhN_km60YruTA,me
44,Ga2Bt7xfqoggTypWD5VpoQ,la
73,0QjROMVW9ACKjhSEfHqNCQ,as
94,Ir_QIzs-4o9ElOtiGuxJrw,la
137,t749-3jKKDyO8QH3VezUHA,as
...,...,...
209315,eUi3O-8Gqh_nJ_ZhO-25gQ,la
209369,5VXlHFrYZZzj7zfZEsz5nQ,as
209376,cmsAFAymfVoBitSF8R6GKw,as
209388,9Q0fPWAjUweoFDk0kafuzQ,as


In [9]:
df = df.merge(res[['business_id', 'restaurant']], how='left')

In [10]:
# narrow this list down to only the restaurants
df = df.dropna(subset = ['restaurant'])
df.head()
len(df)

11856

In [14]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,restaurant,sents
16,bVTjZgRNq8ToxzvtiVrqMA,2hRe26HSCAWbFRn5WChK-Q,d4qwVw4PcN-_2mK2o1Ro1g,1,0,0,0,10pm on a super bowl Sunday and they're alread...,2015-02-02 06:28:00,la,[10pm on a super bowl Sunday and they're alrea...
21,TeX0eOIeZb_wpo86rCedXQ,RR-2nouBn408e3djxC470g,d4qwVw4PcN-_2mK2o1Ro1g,5,0,0,0,Holy heck this place is amazing. I love their ...,2017-02-09 04:25:03,la,"[Holy heck this place is amazing., I love thei..."
24,XvLG7ReC8JZmBltOLJzfcA,-Co-ReNx_lXT1xL_Rr0B2g,XZbuPXdyA0ZtTu3AzqtQhg,4,0,1,0,"As the previous person posted, what more can r...",2009-10-13 09:50:48,am,"[As the previous person posted, what more can ..."
26,-u8Oz_iSZACGwWW_pIT4zw,UreiTV1I9i-XF6_bJhK6Iw,d_q4avZYM1sOlQP7noJKVg,4,0,0,1,Always a fun outing to come for great hot dogs...,2018-04-15 11:13:11,la,[Always a fun outing to come for great hot dog...
47,xFjo0v7PmYWZRI3tK2WPuA,jqEbD-d5iI5nrwhfMmX_5w,8OeHn9KCbICuRBs_XiDDUA,4,1,0,0,Happy to give a strong recommendation to this ...,2013-06-24 04:49:52,am,[Happy to give a strong recommendation to this...


In [11]:
# the star count is saved as a string, let's make it an integer
df['stars'] = df['stars'].map(lambda x: int(x))

Since I'm working with individual sentences, I need to break the reviews down. However since each sentence is associated with a review and each review with a business, I need to make a new DF.

In [12]:
# adding a tokenized sentence column to the dataframe
df['sents'] = df['text'].map(lambda x: nltk.sent_tokenize(x))

# building a new datafram from this
sent1 = []
sent2 = []

for i in df['review_id'][:1000]:
	sent1.append([i, list(df[df['review_id'] == i]['sents'])])

sent1 = [[x, y[0]] for [x,y] in sent1]

for i in sent1:
	counter = 1
	for x in i[1]:
		sent2.append([i[0], counter, x])
		counter = counter + 1

sent_df = pd.DataFrame(sent2)

In [13]:
sent_df

,0,1,2
0,bVTjZgRNq8ToxzvtiVrqMA,1,10pm on a super bowl Sunday and they're alread...
1,bVTjZgRNq8ToxzvtiVrqMA,2,"Weak, no wonder the hard Rock is dying off..."
2,TeX0eOIeZb_wpo86rCedXQ,1,Holy heck this place is amazing.
3,TeX0eOIeZb_wpo86rCedXQ,2,I love their chicken tacos they're by far my f...
4,TeX0eOIeZb_wpo86rCedXQ,3,Great customer service and all round awesome e...
...,...,...,...
8021,jA_AiBeFyYJa-hy9UwPucg,3,I haven't had other than the beer-and-vibe-and...
8022,5v9gIPCuzioHW4QsNYRlkw,1,An absolute must if you like superb & unique t...
8023,5v9gIPCuzioHW4QsNYRlkw,2,Add to that awesome bartenders to great chefs ...
8024,5v9gIPCuzioHW4QsNYRlkw,3,I highly recommend sitting at the bar - it's l...
